In [1]:
from sklearn.linear_model import LogisticRegression
import pickle
import pandas as pd
from sklearn.metrics import average_precision_score,roc_auc_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import auc
from sklearn.ensemble import RandomForestClassifier
from scipy.stats import zscore
from sklearn.preprocessing import normalize
import matplotlib.pyplot as plt
import numpy as np

In [2]:
def train_lr(chrs ,X ,Y ,df ,name ,encode_features,extra_features=None,e2g_fea=True,model_type='lr'):   
    assert model_type in ['lr','rf']
    idx = np.arange(len(Y))   
    X = np.log(np.abs(X) + 0.01)
    
    X_e2g=np.array(X.loc[:,encode_features])
    if extra_features is not None:
        if e2g_fea:
            X_e2g=np.concatenate((X_e2g,extra_features),axis=1)
        else:
            X_e2g=extra_features
    
    print(X_e2g.shape)
    
    for chr in chrs:
        idx_test = df[df['chrom' ]==chr].index.values
        if len(idx_test) > 0:
            idx_train = np.delete(idx, idx_test)

            X_test = X_e2g[idx_test]
            Y_test = Y[idx_test]
            X_train = X_e2g[idx_train, :]
            Y_train = Y[idx_train]
                     
            if model_type=='lr':
                clf = LogisticRegression(random_state=0, class_weight=None, solver='lbfgs',
                                         max_iter=100000).fit(X_train, Y_train)
            else:
                clf = RandomForestClassifier(n_jobs=-1, random_state=0).fit(X_train, Y_train)
            probs = clf.predict_proba(X_test)
            df.loc[idx_test, name +'.Score'] = probs[: ,1]

### Load representations and predictions

In [3]:
reps=np.load('data/re2g_1dreps.npy')
twodreps=np.load('data/re2g_2dreps.npy')
twodreps.shape
with open('data/re2g_wok562_pred.pickle','rb') as f:
    pp=pickle.load(f)



features=[ 'rna','bru', 'groseq', 'grocap', 'netcage','microc','intacthic']

e_extra_fea= np.concatenate([pp['enh'][fea] for fea in features],axis=-1)
p_extra_fea= np.concatenate([pp['tss'][fea] for fea in features],axis=-1)


micro=pp['enh']['microc']
eepis=pp['enh']['epi']
pepis=pp['tss']['epi']
intact=pp['enh']['intacthic']
egros=pp['enh']['groseq']
ebrus=pp['enh']['bru']
enetcage=pp['enh']['netcage']
pgros=pp['tss']['groseq']
pbrus=pp['tss']['bru']
pnetcage=pp['tss']['netcage']
p_extra_fea.shape

(6461, 18)

### Reproduce rE2G

In [7]:
def encode_e2g(extra_fea=None,model_type='lr'): 
    assert model_type in ['lr','rf']

    chr_lens = [248956422, 242193529, 198295559, 190214555, 181538259, 170805979, 159345973, 145138636, 138394717,
                133797422, 135086622, 133275309, 114364328, 107043718, 101991189, 90338345, 83257441, 80373285,
                58617616, 64444167, 46709983, 50818468, 156040895]
    chrs = ['chr' + str(i) for i in range(1, 23)] + ['chrX']
    
    df = pd.read_csv('ENCODE-E2G_Predictions.tsv',sep='\t')
    feature_table = pd.read_csv('model_ablation_encode_e2g.tsv', delimiter = '\t')
    feature_list =feature_table['features']
    df['TSS_bin'] = df['TSS_from_universe'] // 1000
    df['enh_bin'] = ((df['chromStart'] + df['chromEnd']) / 2) // 1000
    df['distance'] = np.abs(df['TSS_from_universe'] - (df['chromStart'] + df['chromEnd']) / 2)
    df['distance'] = np.abs(df['TSS_bin'] - df['enh_bin'])
       
    X1 = df.loc[:, feature_list]
    Y = df['Regulated'].values.astype(np.int64)
    print(Y.sum())
    name1='e2g_gen'
    
 
        
    train_lr(chrs, X1, Y, df, name1,feature_list,
             extra_features=extra_fea,model_type=model_type,e2g_fea=True)
    preds=df[name1 +'.Score'].values.astype(float)

    precision, recall, thresholds = precision_recall_curve(Y,preds)
    aupr = auc(recall, precision)
    idx_recall_70_pct = np.argsort(np.abs(recall - 0.7))[0]
    recall_at_70_pct = recall[idx_recall_70_pct]
    precision_at_70_pct_recall = precision[idx_recall_70_pct]
    threshod_in_70_pct_recall = thresholds[idx_recall_70_pct]
    print(aupr,precision_at_70_pct_recall, threshod_in_70_pct_recall)
    return precision, recall, precision_at_70_pct_recall
precision_base, recall_base, pr70_base=encode_e2g()

472
(10375, 13)
0.6365077696015278 0.541871921182266 0.19498045631554092


### Incorprating general model features to rE2G

In [10]:
def encode_e2g_fea(extra_fea=None,model_type='lr'): 
    assert model_type in ['lr','rf']

    chr_lens = [248956422, 242193529, 198295559, 190214555, 181538259, 170805979, 159345973, 145138636, 138394717,
                133797422, 135086622, 133275309, 114364328, 107043718, 101991189, 90338345, 83257441, 80373285,
                58617616, 64444167, 46709983, 50818468, 156040895]
    chrs = ['chr' + str(i) for i in range(1, 23)] + ['chrX']
    
    df = pd.read_csv('ENCODE-E2G_Predictions.tsv',sep='\t')
    feature_table = pd.read_csv('model_ablation_encode_e2g.tsv', delimiter = '\t')
    feature_list =feature_table['features']
    df['TSS_bin'] = df['TSS_from_universe'] // 1000
    df['enh_bin'] = ((df['chromStart'] + df['chromEnd']) / 2) // 1000
    df['distance'] = np.abs(df['TSS_from_universe'] - (df['chromStart'] + df['chromEnd']) / 2)
    df['distance'] = np.abs(df['TSS_bin'] - df['enh_bin'])
    
    ## keep gene-element pairs within 500kb
    df = df[df['distance'] < 500]
    df = df.reset_index(drop=True)
    df['mid'] = (df['TSS_bin'] + df['enh_bin']) // 2
    df['seqStart_bin'] = df['mid'] - 249
    df['seqEnd_bin'] = df['mid'] + 251
    category_multiplier = dict(zip(chrs, chr_lens))
    df['chrom_len'] = df['chrom'].map(category_multiplier) // 1000
    df = df[df['seqStart_bin'] - 50 > 0]
    df = df[df['seqEnd_bin'] + 50 < df['chrom_len']]
    df = df.reset_index(drop=True)

    
    X1 = df.loc[:, feature_list]
    Y = df['Regulated'].values.astype(np.int64)
    print(Y.sum())
    name1='e2g_gen'
       
    train_lr(chrs, X1, Y, df, name1,feature_list,
             extra_features=extra_fea,model_type=model_type,e2g_fea=True)
    preds=df[name1 +'.Score'].values.astype(float)

    precision, recall, thresholds = precision_recall_curve(Y,preds)
    aupr = auc(recall, precision)
    idx_recall_70_pct = np.argsort(np.abs(recall - 0.7))[0]
    recall_at_70_pct = recall[idx_recall_70_pct]
    precision_at_70_pct_recall = precision[idx_recall_70_pct]
    threshod_in_70_pct_recall = thresholds[idx_recall_70_pct]
    print(aupr,precision_at_70_pct_recall, threshod_in_70_pct_recall)
    return precision, recall, precision_at_70_pct_recall

In [11]:
### rE2G baseline
precision_base, recall_base, pr70_base=encode_e2g_fea()

451
(6461, 13)
0.6573304978706874 0.5693693693693693 0.2247123820655696


In [12]:
## adding predicted histones
epii=np.arange(236,247,1)
extra_fea=np.concatenate((eepis[:,epii],pepis[:,epii]),axis=1)
precision_histone, recall_histone, pr70_histone=encode_e2g_fea(extra_fea)

451
(6461, 35)
0.7236350857612533 0.6583333333333333 0.29713255951581957


In [13]:
## including multiple predicted modalities
epii=np.concatenate((np.array([15]),np.arange(236,247,1)))
extra_fea=np.concatenate((eepis[:,epii],pepis[:,epii],e_extra_fea,p_extra_fea ),axis=1)
precision_allfea, recall_allfea, pr70_allfea=encode_e2g_fea(extra_fea)

451
(6461, 73)
0.767188276425599 0.7165532879818595 0.40963670637209865


In [15]:
### Incorporating general representations
extra_fea=np.concatenate((zscore(reps[:,0,:],axis=1),zscore(reps[:,1,:],axis=1), zscore(twodreps,axis=1)),axis=1)
precision_rep, recall_rep, pr70_rep=encode_e2g_fea(extra_fea,model_type='rf')

451
(6461, 2029)
0.763778110567106 0.7308584686774942 0.42


### Reproduce rE2G-extend

In [20]:
def encode_e2g_extend():
    chr_lens = [248956422, 242193529, 198295559, 190214555, 181538259, 170805979, 159345973, 145138636, 138394717,
                133797422, 135086622, 133275309, 114364328, 107043718, 101991189, 90338345, 83257441, 80373285,
                58617616, 64444167, 46709983, 50818468, 156040895]
    chrs = ['chr' + str(i) for i in range(1, 23)] + ['chrX']
    
    df = pd.read_csv('ENCODE-E2G_Extended_Predictions.tsv',sep='\t')
    df1 = pd.read_csv('ENCODE-E2G_Predictions.tsv',sep='\t')
    df['TSS_from_universe']=df1['TSS_from_universe']
    feature_table = pd.read_csv('model_ablation_encode_e2g_extended.tsv', delimiter = '\t')
    feature_list =feature_table['features']
    
    X1 = df.loc[:, feature_list]
    Y = df['Regulated'].values.astype(np.int64)
    print(Y.sum())
    name1='e2g_extend'
     

    extra_fea=None

 
    train_lr(chrs, X1, Y, df, name1,feature_list,
             extra_features=extra_fea,model_type='lr',e2g_fea=True)
    preds=df[name1 +'.Score'].values.astype(float)

    precision, recall, thresholds = precision_recall_curve(Y,preds)
    aupr = auc(recall, precision)
    idx_recall_70_pct = np.argsort(np.abs(recall - 0.7))[0]
    recall_at_70_pct = recall[idx_recall_70_pct]
    precision_at_70_pct_recall = precision[idx_recall_70_pct]
    threshod_in_70_pct_recall = thresholds[idx_recall_70_pct]
    print(aupr,precision_at_70_pct_recall, threshod_in_70_pct_recall)
    return precision, recall, precision_at_70_pct_recall
precision_extend, recall_extend, pr70_extend=encode_e2g_extend()   

472
(10375, 46)
0.7591947086239141 0.7066381156316917 0.31996124119169


In [23]:
def encode_e2g_extend_fea(extra_fea=None,model_type='lr'): 
    assert model_type in ['lr','rf']
    chr_lens = [248956422, 242193529, 198295559, 190214555, 181538259, 170805979, 159345973, 145138636, 138394717,
                133797422, 135086622, 133275309, 114364328, 107043718, 101991189, 90338345, 83257441, 80373285,
                58617616, 64444167, 46709983, 50818468, 156040895]
    chrs = ['chr' + str(i) for i in range(1, 23)] + ['chrX']
    
    df = pd.read_csv('ENCODE-E2G_Extended_Predictions.tsv',sep='\t')
    df1 = pd.read_csv('ENCODE-E2G_Predictions.tsv',sep='\t')
    df['TSS_from_universe']=df1['TSS_from_universe']
    feature_table = pd.read_csv('model_ablation_encode_e2g_extended.tsv', delimiter = '\t')
    feature_list =feature_table['features']
    
    df['TSS_bin'] = df['TSS_from_universe'] // 1000
    df['enh_bin'] = ((df['chromStart'] + df['chromEnd']) / 2) // 1000
    df['distance'] = np.abs(df['TSS_from_universe'] - (df['chromStart'] + df['chromEnd']) / 2)
    df['distance'] = np.abs(df['TSS_bin'] - df['enh_bin'])
    
    df = df[df['distance'] < 500]
    df = df.reset_index(drop=True)
    df['mid'] = (df['TSS_bin'] + df['enh_bin']) // 2
    df['seqStart_bin'] = df['mid'] - 249
    df['seqEnd_bin'] = df['mid'] + 251
    category_multiplier = dict(zip(chrs, chr_lens))
    df['chrom_len'] = df['chrom'].map(category_multiplier) // 1000
    df = df[df['seqStart_bin'] - 50 > 0]
    df = df[df['seqEnd_bin'] + 50 < df['chrom_len']]
    df = df.reset_index(drop=True)


    X1 = df.loc[:, feature_list]
    Y = df['Regulated'].values.astype(np.int64)
    print(Y.sum())
    name1='e2g_extend'
    
 
    train_lr(chrs, X1, Y, df, name1,feature_list,
             extra_features=extra_fea,model_type=model_type,e2g_fea=True)
    preds=df[name1 +'.Score'].values.astype(float)

    precision, recall, thresholds = precision_recall_curve(Y,preds)
    aupr = auc(recall, precision)
    idx_recall_70_pct = np.argsort(np.abs(recall - 0.7))[0]
    recall_at_70_pct = recall[idx_recall_70_pct]
    precision_at_70_pct_recall = precision[idx_recall_70_pct]
    threshod_in_70_pct_recall = thresholds[idx_recall_70_pct]
    print(aupr,precision_at_70_pct_recall, threshod_in_70_pct_recall)
    return precision, recall, precision_at_70_pct_recall

In [24]:
precision_extend, recall_extend, pr70_extend=encode_e2g_extend_fea()  

451
(6461, 46)
0.7852553430986395 0.7400468384074942 0.38703962122655555


In [26]:
# Including predicted features
extra_fea=np.concatenate((e_extra_fea,p_extra_fea),axis=1)
precision_allfea, recall_allfea, pr70_allfea=encode_e2g_extend_fea(extra_fea)

451
(6461, 82)
0.8003000379154435 0.7669902912621359 0.4585920980137191


In [27]:
### Incorporating general representations
extra_fea=np.concatenate((zscore(reps[:,0,:],axis=1),zscore(reps[:,1,:],axis=1), zscore(twodreps,axis=1)),axis=1)
precision_extra_rep, recall_extra_rep, pr70_extra_rep=encode_e2g_extend_fea(extra_fea,model_type='rf')

451
(6461, 2062)
0.7687511953535031 0.7394366197183099 0.42
